In [4]:
import pandas as pd
from Bio import SeqIO
from Bio.ExPASy import ScanProsite

So I dun goofed and forgot to have you you guys download sequences as well for this next step, below is a script you help you merge the two files automatically, you'll use a similar pattern for the next set of steps which invovles extracting protien domains. 

In [5]:
#Read your blast results csv in
df = pd.read_csv("C:\\Users\\cmgro\\Downloads\\gadE_genome_feature (3).csv")
#Show the first 5 rows
print(df.head())
#Temp lists to attatch variables 
ids = []
seqs = []
#Seq.IO.parse is. an easy way to read through every gene/prot in a fasta file 
for record in SeqIO.parse("C:\\Users\\cmgro\\Downloads\\gadE_genome_feature (3).fasta", "fasta"):
    # So the fasta ID line is fig|1240239.3.peg.2328|XFF4834R_chr22840|VBIXanFus78442_2328|
    # but we only need the first two elements ("fig and 1240239.3.peg.2328") to make fig|1240239.3.peg.2328
    # We then use this ID to merge back onto the original dataframe (basically we say if this row has this ID, add the sequence to that row)
    
    #Fancy way of extracting the ID
    ids.append("|".join(record.id.split("|")[0:2]))
    #Fancy way of extracting the sequence 
    seqs.append(str(record.seq))
#making a new dataframe from this info 
seq_df = pd.DataFrame.from_dict({"PATRIC ID": ids, "Sequences":seqs})
print(seq_df.head())
#Mergine back and saving 
df = df.merge(seq_df, on="PATRIC ID")
df.to_csv("/Users/cmgro/Downloads/merged_gadE.csv")

                                     Genome  Genome ID     Accession  \
0         Yokenella regensburgei ATCC 43003  1002368.3  AGCL01000039   
1  Escherichia coli O104:H4 str. 2011C-3493  1133852.3      CP003289   
2  Escherichia coli O104:H4 str. 2011C-3493  1133852.3      CP003289   
3         Kluyvera cryocrescens NBRC 102467  1218112.3  BCTM01000004   
4             Shigella flexneri 2a str. 301   198214.7     NC_004337   

                PATRIC ID  RefSeq Locus Tag         Alt Locus Tag  \
0  fig|1002368.3.peg.2680  HMPREF0880_02941  VBIYokReg189340_2680   
1   fig|1133852.3.peg.277         O3K_01350  VBIEscCol234630_0277   
2   fig|1133852.3.peg.282         O3K_01375  VBIEscCol234630_0282   
3   fig|1218112.3.peg.722               NaN                   NaN   
4   fig|198214.7.peg.4170            SF3541   VBIShiFle86970_4170   

                                          Feature ID Annotation Feature Type  \
0  PATRIC.1002368.3.AGCL01000039.CDS.217786.21831...     PATRIC         

In [6]:
import time
sigs = []

for i in df["Sequences"].values:
    handle = ScanProsite.scan(seq=i, mirror="https://prosite.expasy.org")
    result = ScanProsite.read(handle)
    qq = []
    for i in result:
        print(i)
        qq.append(i["signature_ac"])
    time.sleep(5)
    qq = set(qq)
    sigs.append(qq)
    
df["Prosite results"] = sigs
df.to_csv("/Users/christine/Downloads/gadE_motifs.csv")

{'sequence_ac': 'USERSEQ1', 'start': 110, 'stop': 175, 'signature_ac': 'PS50043', 'score': '13.775', 'level': '0'}
{'sequence_ac': 'USERSEQ1', 'start': 109, 'stop': 174, 'signature_ac': 'PS50043', 'score': '20.766', 'level': '0'}
{'sequence_ac': 'USERSEQ1', 'start': 109, 'stop': 174, 'signature_ac': 'PS50043', 'score': '25.818', 'level': '0'}
{'sequence_ac': 'USERSEQ1', 'start': 110, 'stop': 175, 'signature_ac': 'PS50043', 'score': '14.686', 'level': '0'}
{'sequence_ac': 'USERSEQ1', 'start': 100, 'stop': 165, 'signature_ac': 'PS50043', 'score': '25.994', 'level': '0'}
{'sequence_ac': 'USERSEQ1', 'start': 109, 'stop': 174, 'signature_ac': 'PS50043', 'score': '20.766', 'level': '0'}
{'sequence_ac': 'USERSEQ1', 'start': 109, 'stop': 174, 'signature_ac': 'PS50043', 'score': '19.767', 'level': '0'}
{'sequence_ac': 'USERSEQ1', 'start': 100, 'stop': 165, 'signature_ac': 'PS50043', 'score': '25.994', 'level': '0'}
{'sequence_ac': 'USERSEQ1', 'start': 100, 'stop': 165, 'signature_ac': 'PS50043'

FileNotFoundError: [Errno 2] No such file or directory: '/Users/christine/Downloads/gadE_motifs.csv'

In [ ]:
df
